# 第五章数据处理与加载

在 PyTorch 中，处理和加载数据是深度学习训练过程中的关键步骤。

为了高效地处理数据，PyTorch 提供了强大的工具，包括 torch.utils.data.Dataset 和 torch.utils.data.DataLoader，帮助我们管理数据集、批量加载和数据增强等任务。

PyTorch 数据处理与加载的介绍：

自定义 Dataset：通过继承 torch.utils.data.Dataset 来加载自己的数据集。
DataLoader：DataLoader 按批次加载数据，支持多线程加载并进行数据打乱。
数据预处理与增强：使用 torchvision.transforms 进行常见的图像预处理和增强操作，提高模型的泛化能力。
加载标准数据集：torchvision.datasets 提供了许多常见的数据集，简化了数据加载过程。
多个数据源：通过组合多个 Dataset 实例来处理来自不同来源的数据。

自定义 Dataset
torch.utils.data.Dataset 是一个抽象类，允许你从自己的数据源中创建数据集。

我们需要继承该类并实现以下两个方法：

__len__(self)：返回数据集中的样本数量。
__getitem__(self, idx)：通过索引返回一个样本。

In [145]:
import torch
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self,X_data,Y_data):
        """
        初始化数据集，xdata和ydata是两个列表或者数组
        x是输入特征
        y是目标标签
        """
        self.X_data=X_data
        self.Y_data=Y_data

    def __len__(self):
        return len(self.X_data)

    def __getitem__(self,idx):
        x=torch.tensor(self.X_data[idx],dtype=torch.float32)
        y=torch.tensor(self.Y_data[idx],dtype=torch.float32)
        return x,y

# 示例数据
X_data = [[1, 2], [3, 4], [5, 6], [7, 8]]  # 输入特征
Y_data = [1, 0, 1, 0]  # 目标标签

# 创建数据集实例
dataset = MyDataset(X_data, Y_data)


使用dataloader加载数据，用于从dataset中按照批次batch加载数据
批量读取数据进行多线程加载，从而提高训练效率


In [2]:
from torch.utils.data import DataLoader

dataloader= DataLoader(dataset,batch_size=2,shuffle=True)

for epoch in range(1):
    for batch_idx,(inputs,labels) in enumerate(dataloader):
        print(f'Batch {batch_idx + 1}:')
        print(f'Inputs: {inputs}')
        print(f'Labels: {labels}')  
#batch_idx放置在(inputs, labels)后面会报错是因为enumerate()函数的返回值是一个包含两个元素的元组：
        # 第一个元素是索引（即batch_idx），
        # 第二个元素是数据（在这种情况下是(inputs, labels)）。

NameError: name 'dataset' is not defined

batch_size: 每次加载的样本数量。
shuffle: 是否对数据进行洗牌，通常训练时需要将数据打乱。
drop_last: 如果数据集中的样本数不能被 batch_size 整除，设置为 True 时，丢弃最后一个不完整的 batc
每次循环中，DataLoader 会返回一个批次的数据，包括输入特征（inputs）和目标标签（labels）h。

## 预处理与数据增强

In [3]:
import torchvision.transforms as transforms
from PIL import Image

transform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
])
image=Image.open('image.jpg')

image_tensor=transform(image)
print(image_tensor.shape)
image_tensor
'''
transforms.Compose()：将多个变换操作组合在一起。
transforms.Resize()：调整图像大小。
transforms.ToTensor()：将图像转换为 PyTorch 张量，值会被归一化到 [0, 1] 范围。
transforms.Normalize()：标准化图像数据，通常使用预训练模型时需要进行标准化处理。
图像数据增强
'''

torch.Size([3, 128, 128])


'\ntransforms.Compose()：将多个变换操作组合在一起。\ntransforms.Resize()：调整图像大小。\ntransforms.ToTensor()：将图像转换为 PyTorch 张量，值会被归一化到 [0, 1] 范围。\ntransforms.Normalize()：标准化图像数据，通常使用预训练模型时需要进行标准化处理。\n图像数据增强\n'

In [154]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 随机水平翻转
    transforms.RandomRotation(30),  # 随机旋转 30 度
    transforms.RandomResizedCrop(128),  # 随机裁剪并调整为 128x128
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# 定义预处理操作
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 对灰度图像进行标准化
])

# 下载并加载 MNIST 数据集
train_dataset = datasets.MNIST(root='data', train=True, download=False, transform=transform)
test_dataset = datasets.MNIST(root='data', train=False, download=False, transform=transform)

# 创建 DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 迭代训练数据
for inputs, labels in train_loader:
    print(inputs.shape)  # 每个批次的输入数据形状
    print(labels.shape)  # 每个批次的标签形状

torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size([64, 1, 28, 28])
torch.Size([64])
torch.Size

## 用多个数据源（Multi-source Dataset）

如果你的数据集由多个文件、多个来源（例如多个图像文件夹）组成，可以通过继承 Dataset 类自定义加载多个数据源。

PyTorch 提供了 ConcatDataset 和 ChainDataset 等类来连接多个数据集。

例如，假设我们有多个图像文件夹的数据，可以将它们合并为一个数据集：